In [2]:
import cv2
import numpy as np
import os
import torch
from simple_hrnet import HRNet

ModuleNotFoundError: No module named 'simple_hrnet'

In [9]:
def load_yolo_model(coco_names_path):
    # Load YOLO model
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open(coco_names_path, "r") as f:
        classes = [line.strip() for line in f.readlines()]

    # Get the names of the output layers
    output_layers = net.getUnconnectedOutLayersNames()

    return net, classes, output_layers

In [10]:
def load_hrnet_model(model_path):
    # Initialize HRNet model with the number of keypoints
    num_keypoints = 17  # Number of keypoints for COCO dataset
    model = HRNet(num_keypoints=num_keypoints)

    # Load pre-trained weights
    model.load_state_dict(torch.load(model_path))
    
    # Set the model to evaluation mode
    model.eval()

    return model

In [11]:
def estimate_pose(image, model):
    # Perform pose estimation using HRNet model (replace with actual pose estimation code)
    # ...
    # Return pose keypoints (x, y) as a list of tuples
    pose_keypoints = [(x, y), ...]
    return pose_keypoints

In [12]:
'''def detect_objects(image_path, net, classes, output_layers, confidence_threshold=0.5, nms_threshold=0.3):
    # Load image
    image = cv2.imread(image_path)
    height, width, channels = image.shape

    # Preprocess image for YOLO
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Post-process detections
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Apply Non-maximum Suppression to remove duplicate detections
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

    # Convert indices to a 1D array using np.squeeze
    indices = np.squeeze(indices)

    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    for i in indices:
        box = boxes[i]
        x, y, w, h = box
        class_id = class_ids[i]
        label = f"{classes[class_id]} {confidences[i]:.2f}"
        color = colors[class_id]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Show the image with detections
    cv2.imshow("Object Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()'''

def detect_objects(image_path, net, classes, output_layers, confidence_threshold=0.5, nms_threshold=0.3):
    # Load image
    image = cv2.imread(image_path)
    height, width, channels = image.shape

    # Preprocess image for YOLO
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    # Post-process detections
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Apply Non-maximum Suppression to remove duplicate detections
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)

    # Convert indices to a 1D array using np.squeeze
    indices = np.squeeze(indices)

    # Check if indices is not empty before proceeding with the iteration
    if indices.ndim == 0:
        indices = np.expand_dims(indices, axis=0)

    colors = np.random.uniform(0, 255, size=(len(classes), 3))
    for i in indices:
        box = boxes[i]
        x, y, w, h = box
        class_id = class_ids[i]
        label = f"{classes[class_id]} {confidences[i]:.2f}"
        color = colors[class_id]
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
        cv2.putText(image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    cv2.imshow("Object Detection", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [ ]:
if __name__ == "__main__":
    coco_names_path = "C:/Users/Dhriti Shah/Downloads/coco.names"
    net, classes, output_layers = load_yolo_model(coco_names_path)

    image_folder = 'C:/Users/Dhriti Shah/Downloads/archive/var_200/VAR/Fouls'
    for filename in os.listdir(image_folder):
        if filename.endswith('.jpg'):
            image_path = os.path.join(image_folder, filename)
            detect_objects(image_path, net, classes, output_layers)